In [ ]:
from lxml import html
import requests #For making HTTP requests to URLS.
import pandas as pd
from IPython.display import display, Image
import re

Full Function

In [28]:
# Function to parse the html receipt and return a dataframe with the receipt information
def parse_html_receipt(html_path):
    # Read the local HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Parse the HTML content using lxml
    parsed_html_receipt = html.fromstring(file_content)

    # Extract all class attributes from the HTML elements
    class_attributes = parsed_html_receipt.xpath('//@class')

    # obtain unique kinds of delivered items
    delivered_item_tags = []
    for cls in class_attributes:
        if 'item-row item-delivered' in cls:
            if cls not in delivered_item_tags:
                delivered_item_tags.append(cls)

    # Create a dictionary to store the information of the delivered items        
    items_dict = {'item-name':[],  'item-price':[], 'item-image':[]}
    for delivered_item_tag in delivered_item_tags:
        div_element = parsed_html_receipt.xpath(f'.//div[@class="{delivered_item_tag}"]')
        # Obtain all subelements inside the div element
        for item_div in div_element:
            # print(html.tostring(item_div, encoding='unicode'))
            for class_name in items_dict.keys():
                if class_name == 'item-name': #obtain item name
                    item_name =  item_div.xpath(f'.//div[@class="{class_name}"]')
                    cleaned_text = [div.text_content().replace('\n', '') for div in item_name ]
                    items_dict[class_name].append(cleaned_text[0])
                if class_name == 'item-price': #obtain item price
                    div_elements = item_div.xpath(f'.//div[@class="{class_name}"]')
                    cleaned_prices = [div.text_content().replace('\n', '') for div in div_elements]
                    # print(cleaned_prices)
                    items_dict[class_name].append(float(re.search(r'Final item price:\$(\d+\.\d+)', cleaned_prices[0]).group(1)))
                if class_name == 'item-image': #obtain the image of the item  
                    div_elements = item_div.xpath('.//div[@class="item-image"]/img/@src')
                    items_dict[class_name].append(div_elements[0])
            
    delivered_items_df = pd.DataFrame(items_dict) #DataFrame with the delivered items

    # Obtain the total price of the order and miscellaneous charges
    misc_charges = {'charge-type':[], 'amount':[]}
    charge_type = parsed_html_receipt.xpath('//tr/td[@class="charge-type"]/text()') #extract the charges
    amount_type = parsed_html_receipt.xpath('//tr/td[@class="amount"]/text()') #extract the cost for each charge
    misc_charges['charge-type'] = [c for c in charge_type if c!='Instacart+ Member Free Delivery!'] #remove the free delivery charge
    print(charge_type)
    print(misc_charges['charge-type'])
    misc_charges['amount'] = [float(i.replace('$', '')) for i in amount_type]
    misc_charges_df = pd.DataFrame(misc_charges) #DataFrame with the miscellaneous charges

    #obtain the text conntent of this element
    delivery_details = parsed_html_receipt.xpath('//div[@class="DriverDeliverySchedule"]/text()') #extract the charges
    print(delivery_details[0].replace('\n', ''))
    return delivered_items_df, misc_charges_df

In [29]:
html_path = 'test2.html'
delivered_items_df, misc_charges_df = parse_html_receipt(html_path)


['Items Subtotal', 'Checkout Bag Fee', 'Sales Tax', 'Tip', 'Service Fee', 'Total', 'Instacart+ Member Free Delivery!', 'Original Charge', 'Total Charged']
['Items Subtotal', 'Checkout Bag Fee', 'Sales Tax', 'Tip', 'Service Fee', 'Total', 'Original Charge', 'Total Charged']
Your order from ALDI was placed on February 15th, 2023 and delivered on February 15th, 2023 at 4:26 PM


In [17]:
misc_charges_df.iloc[0]

charge-type    Items Subtotal
amount                  73.22
Name: 0, dtype: object

In [ ]:
# Read the local HTML file
html_path = 'test2.html'

# Read the local HTML file
with open(html_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# Parse the HTML content using lxml
parsed_html_from_html = html.fromstring(file_content)

# Find all table elements
tables = parsed_html_from_html.xpath('//table')
print(len(tables))

for idx, table in enumerate(tables):
    # print(f"Attributes for table {idx + 1}:")
    class_name = table.attrib['class']
    print(f'class for table {idx}: {class_name}')



In [ ]:
# Extract all class attributes from the HTML elements
class_attributes = parsed_html_from_html.xpath('//@class')

# Split and flatten the classes into a list
all_classes = [c.strip() for classes in class_attributes for c in classes.split()]

# Get unique classes by converting the list to a set
unique_classes = set(all_classes)

# Print the unique classes
delivered_item_tags = []
for cls in class_attributes:
    if 'item-row item-delivered' in cls:
        if cls not in delivered_item_tags:
            delivered_item_tags.append(cls)
print(delivered_item_tags)

delivered_items, Replacements, Refunds and adjustments

In [ ]:
#### REPLACEMENTS
# adjustments_table = tables[1]
items_dict = {'item-name':[],  'item-price':[], 'item-image':[]}
for delivered_item_tag in delivered_item_tags:
    div_element = parsed_html_from_html.xpath(f'.//div[@class="{delivered_item_tag}"]')
    # Obtain all subelements inside the div element
    for item_div in div_element:
        # print(html.tostring(item_div, encoding='unicode'))
        for class_name in items_dict.keys():
            if class_name == 'item-name': #obtain item name
                item_name =  item_div.xpath(f'.//div[@class="{class_name}"]')
                cleaned_text = [div.text_content().replace('\n', '') for div in item_name ]
                items_dict[class_name].append(cleaned_text[0])
            if class_name == 'item-price':
                div_elements = item_div.xpath(f'.//div[@class="{class_name}"]')
                cleaned_prices = [div.text_content().replace('\n', '') for div in div_elements]
                # print(cleaned_prices)
                items_dict[class_name].append(float(re.search(r'Final item price:\$(\d+\.\d+)', cleaned_prices[0]).group(1)))
            if class_name == 'item-image':        
                div_elements = item_div.xpath('.//div[@class="item-image"]/img/@src')
                items_dict[class_name].append(div_elements[0])
        
delivered_items_df = pd.DataFrame(items_dict)
for _, row in replaced_items_df.iterrows():
    display(Image(row['item-image']))
    display(row[['item-name', 'item-price']])
    print('\n')




Obtaing the sub-total, extra charges and the total charge.

In [ ]:
sub_total_table = parsed_html_from_html
charge_type = sub_total_table.xpath('//tr/td[@class="charge-type"]/text()') #extract the charges
amount_type = sub_total_table.xpath('//tr/td[@class="amount"]/text()') #extract the cost for each charge
charges = [(charge_type[i], float(amount_type[i].replace('$', ''))) for i in range(len(amount_type))]
print(charges)    

In [ ]:

# Code to obtain unchanged delivered objects.

# delivered_items = tables[2]

# items_dict = {'item-name':[],  'item-price':[], 'item-image':[]}

# for class_name in items_dict.keys():   
#     if class_name == 'item-name': #obtain item name
#         div_elements = delivered_items.xpath(f'.//div[@class="{class_name}"]')
#         cleaned_text = [div.text_content().replace('\n', '') for div in div_elements]
#         items_dict[class_name] = [text.strip() for text in cleaned_text]
    
#     if class_name == 'item-price':
#         div_elements = delivered_items.xpath(f'.//div[@class="{class_name}"]')
#         cleaned_prices = [div.text_content().replace('\n', '') for div in div_elements]
#         items_dict[class_name] = [float(price.split('$')[1]) for price in cleaned_prices]
    
#     if class_name == 'item-image':        
#         div_elements = delivered_items.xpath('.//div[@class="item-image"]/img/@src')
#         items_dict[class_name] = div_elements

# delivered_items_df = pd.DataFrame(items_dict) #DataFrame of items with names, images and prices.
# for _, row in delivered_items_df.iterrows():
#     display(Image(row['item-image']))
#     display(row[['item-name', 'item-price']])
#     print('\n')